# Import dependencies

In [1]:
import io
import pandas as pd
import requests
import re

# Scrape and clean the conference affiliation for each school

In [2]:
# get the teams & conferences
url = "https://en.wikipedia.org/wiki/Power_Five_conferences"

# read in the list of Power 5 conferences (will change ahead of 2024 season)
df = pd.read_html(url)

# select the first dataframe to use
df = df[0]

# remove all asterisks
df = df.apply(lambda x: x.str.replace('*', ''))

# Replace NaN values with blanks (empty strings)
df = df.fillna('')

# Melt the dataframe to unpivot it
p5_df = pd.melt(df, var_name='Conference', value_name='School')

# Drop rows with NaN values in the 'School' column
p5_df = p5_df[p5_df['School'] != '']

# Reset the index
p5_df.reset_index(drop=True, inplace=True)

# show dataframe
p5_df.to_csv('p5_affliation.csv')

print('export complete!')

export complete!


# Get betting lines from thepredictiontracker.com

In [157]:
# first, scrape the 'last updated' date from thepredictiontracker.com
url = "https://www.thepredictiontracker.com/predncaa.html"

# Send a GET request to the URL to fetch the webpage content
response = requests.get(url)

# pattern to find the latest updated date
pattern = r"Updated: (.*?):"

# get the returned text that matches
match = re.search(pattern,response.text)

# just grab the text that includes the day & date
last_updated = match.group(1)

# Remove the final 3 characters
last_updated = last_updated[:-3]

print(f'Last updated: {last_updated}')

Last updated: Wednesday, September 27, 2023


In [132]:
session = requests.Session()
response = session.get('https://www.thepredictiontracker.com/ncaapredictions.csv')

# Create a file-like object from the bytes object
csv_file = io.StringIO(response.content.decode())

# Read the CSV file into a Pandas dataframe
df = pd.read_csv(csv_file)

# Define a subset of column names (all the various servies that put out a betting line) for which we'll find the minimum value
subset_columns = [
    'linehow',
    'lineargh',
    'linecong',
    'linebillings',
    'linepiratings',
    'linepimean',
    'linepibias',
    'linemassey',
    'lineclean',
    'linedonchess',
    'linepugh',
    'linedwig',
    'linefpi',
    'linetalis',
    'lineround',
    'linesag',
    'linesagr',
    'linesagpred',
    'linesaggm',
    'linepfz',
    'linemoore',
    'lineelo',
    'lineborn',
    'linefox',
    'linelaz',
    'linepig',
    'linekam',
    'linepayne',
    'linepaynep',
    'linepaynewl',
    'linekerns',
    'lineteamrank',
    'lineloud',
    'linecurry',
    'linedunk',
    'linewayward'
]

# Define a custom function to find the minimum value among non-null values in a row
def min_non_null(row):
    non_null_values = [value for value in row if not pd.isna(value)]
    if non_null_values:
        return min(non_null_values)
    return np.nan

# create a temporary, subset dataframe using only the columns specified above (these will all be numeric-only columns)
subset_df = df[subset_columns]

# Find the min by applying the custom function along axis=1 to find the minimum non-null value for each row in the subset of columns
df['line_min'] = subset_df.apply(min_non_null, axis=1)

# Find the max by is easy, because we don't have to consider only non-null values. They would never be returned as a max value anyways
df['line_max'] = subset_df.max(axis=1)


# keep the following columns
df = df[[
    'home',
    'road',
    'lineopen',
    'line',
    'linemidweek',
    # 'line_max',
    # 'line_min',
    'lineavg',
    'linestd',
    'linemedian',
    'phcover',
    'phwin'
]]

# Replace "St." with "State" in 'home' and 'road' columns
df['home'] = df['home'].str.replace("St.", "State")
df['road'] = df['road'].str.replace("St.", "State")

# Replace "Va." with "Virginia" in 'home' and 'road' columns
df['home'] = df['home'].str.replace("Va.", "Virginia")
df['road'] = df['road'].str.replace("Va.", "Virginia")

# Replace "Mississippi" with "Ole Miss", which requires a bit more work
def replace_mississippi(value):
    if value == "Mississippi":
        return "Ole Miss"
    else:
        return value

# Apply the custom function to the 'home' and 'road' columns
df['home'] = df['home'].apply(replace_mississippi)
df['road'] = df['road'].apply(replace_mississippi)

# show dataframe
df.head()

,home,road,lineopen,line,linemidweek,lineavg,linestd,linemedian,phcover,phwin
0,Air Force,San Diego State,10.5,10.0,NaN,13.598890,3.728550,12.75878,0.562234,0.796967
1,Akron,Buffalo,1.5,3.0,NaN,1.058622,5.285043,0.10000,0.466769,0.525136
2,Arizona,Washington,-9.5,-18.0,NaN,-19.336034,5.983048,-18.01000,0.477281,0.128010
3,Arkansas,Texas A&M,-6.5,-6.5,NaN,-7.316401,4.798616,-7.00000,0.485948,0.330121
4,Auburn,Georgia,-15.5,-14.5,NaN,-11.589624,5.106635,-11.00000,0.549831,0.244307


# Filter for P5 matchups & clean dataset

In [169]:
# Looking at the 'home' or 'road' columns
filtered_df = df[df['home'].isin(p5_df['School']) | df['road'].isin(p5_df['School'])]

# Reset the index if needed
filtered_df.reset_index(drop=True, inplace=True)

# Merge 'p5_df' into 'df' based on the "School" and "home" columns
merged_df = filtered_df.merge(p5_df[['School', 'Conference']], left_on='home', right_on='School', how='left')

# Rename the "Conference" column for home teams
merged_df = merged_df.rename(columns={'Conference': 'homeTeam_conference'})

# Merge 'p5_df' into 'df' again for the "road" column
merged_df = merged_df.merge(p5_df[['School', 'Conference']], left_on='road', right_on='School', how='left')

# Rename the "Conference" column for road teams
merged_df = merged_df.rename(columns={'Conference': 'roadTeam_conference'})

# Drop the extra "School" columns
merged_df = merged_df.drop(columns=['School_x', 'School_y'])

# fill in the teams that aren't in the Power 5
merged_df['homeTeam_conference'] = merged_df['homeTeam_conference'].fillna('Non-P5')
merged_df['roadTeam_conference'] = merged_df['roadTeam_conference'].fillna('Non-P5')

# if not mid-week line has been released, just fill this in
merged_df['linemidweek'] = merged_df['linemidweek'].fillna('no line yet')

# create a column to show movement in the lines
merged_df['line_movement'] = abs(merged_df['line'] - merged_df['lineopen'])

# create matchup column
merged_df['matchup'] = merged_df['road'] + ' @ ' + merged_df['home']

# function to create string of betting favorite at OPENING line
def opening_line(row):
    if row['lineopen'] < 0:
        return f"{row['road']} {row['lineopen']}"
    else:
        return f"{row['home']} {-row['lineopen']}"

# Apply the custom function to create the new column
merged_df['opening_line'] = merged_df.apply(opening_line, axis=1)

# function to create string of betting favorite at CURRENT line
def current_line(row):
    if row['line'] < 0:
        return f"{row['road']} {row['line']}"
    else:
        return f"{row['home']} {-row['line']}"

# Apply the custom function to create the new column
merged_df['current_line'] = merged_df.apply(current_line, axis=1)

# function to show the average prediction outcome
def average_prediction_outcome(row):
    if row['lineavg'] < 0:
        return f"{row['road']} by {-row['lineavg']:.1f}"
    else:
        return f"{row['home']} by {row['lineavg']:.1f}"

# Apply the custom function to create the new column
merged_df['avg_predicted_winner'] = merged_df.apply(average_prediction_outcome, axis=1)

# drop unneeded columns
merged_df = merged_df.drop([
    'lineopen',
    'line',
    'lineavg',
    'linemedian'
], axis=1)

# rename
merged_df = merged_df.rename(columns={
    'linestd':'prediction_st_dev',
    'phcover':'prob_homeTeam_covers',
    'phwin':'prob_homeTeam_wins',
    'linemidweek':'midweek_line'
})

merged_df['prob_roadTeam_covers'] = 1 - merged_df['prob_homeTeam_covers'] 

# rearrange columns
merged_df = merged_df[[
    'home',
    'road',
    'matchup',
    'homeTeam_conference',
    'roadTeam_conference',
    'opening_line',
    'current_line',
    'line_movement',
    'midweek_line',
    'prob_homeTeam_covers',
    'prob_roadTeam_covers',
    'avg_predicted_winner',
    'prediction_st_dev',
    'prob_homeTeam_wins'
]]

merged_df['brinks_number'] = merged_df.apply(lambda row: max(row['prob_homeTeam_covers'], row['prob_roadTeam_covers']), axis=1)

# Create a new 'brinks_label' column based on the greater value
merged_df['brinks_label'] = merged_df.apply(
    lambda row: f"{row['home']} covers" if row['prob_homeTeam_covers'] > row['prob_roadTeam_covers'] else f"{row['road']} covers",
    axis=1
)

# sort values by highest liklihood of either home or road team covering
merged_df.sort_values(by='brinks_number', ascending=False).head()

,home,road,matchup,homeTeam_conference,roadTeam_conference,opening_line,current_line,line_movement,midweek_line,prob_homeTeam_covers,prob_roadTeam_covers,avg_predicted_winner,prediction_st_dev,prob_homeTeam_wins,brinks_number,brinks_label
23,Syracuse,Clemson,Clemson @ Syracuse,ACC,ACC,Clemson -6.5,Clemson -7.0,0.5,no line yet,0.639745,0.360255,Syracuse by 1.3,5.016475,0.531189,0.639745,Syracuse covers
7,Colorado,USC,USC @ Colorado,Pac-12,Pac-12,USC -27.5,USC -21.5,6.0,no line yet,0.611525,0.388475,USC by 14.7,7.975713,0.204869,0.611525,Colorado covers
14,Ole Miss,LSU,LSU @ Ole Miss,SEC,SEC,LSU -2.0,LSU -2.5,0.5,no line yet,0.592880,0.407120,Ole Miss by 2.9,3.635048,0.570284,0.592880,Ole Miss covers
9,Georgia Tech,Bowling Green,Bowling Green @ Georgia Tech,ACC,Non-P5,Georgia Tech -20.5,Georgia Tech -21.5,1.0,no line yet,0.416914,0.583086,Georgia Tech by 16.7,4.267201,0.843700,0.583086,Bowling Green covers
3,BYU,Cincinnati,Cincinnati @ BYU,Big 12,Big 12,BYU -2.5,Cincinnati -2.5,5.0,no line yet,0.568412,0.431588,BYU by 1.5,4.893592,0.535775,0.568412,BYU covers
